### Вариант 1
Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
<br>1. Наименование вакансии.
<br>2. Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
<br>3. Ссылку на саму вакансию.
<br>4. Сайт, откуда собрана вакансия.
<br>По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

In [78]:
from bs4 import BeautifulSoup as bs
import requests
from lxml import html
import re

### Начало работы с парсингом сайта hh.ru

In [1200]:
url = 'https://hh.ru/search/vacancy'
headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
            }

In [1201]:
# <div class="pager" data-qa="pager-block"></div> - блок переключения страниц с найдеными вакансиями
# если вакансии умещаются на одной странице то мы их спарсим, если же страниц больше, то парсер упустит вакансии на других
# страницах, поэтому нужно задать условие, чтобы получить количество страниц и парсить каждую страницу

In [1202]:
# <input type="text" name="text" autocomplete="off" data-qa="vacancysearch__keywords-input" aria-autocomplete="list" class="bloko-input-text" value="">
# ввод ключевых слов для поиска вакансии
# key: text
# value: vacancy_name (задаем в поиске, например: Разработчик Java)

In [1203]:
# <input type="checkbox" name="search_field" value="name" data-qa="control-vacancysearch__search_field-item control-vacancysearch__search_field-item_name" class="bloko-checkbox__input">
# чек бокс поиска ключевого слова в названии вакансии
# <input type="checkbox" name="search_field" value="company_name" data-qa="control-vacancysearch__search_field-item control-vacancysearch__search_field-item_company_name" class="bloko-checkbox__input">
# чек бокс поиска ключевого слова в названии компании
# <input type="checkbox" name="search_field" value="description" data-qa="control-vacancysearch__search_field-item control-vacancysearch__search_field-item_description" class="bloko-checkbox__input">
# чек бокс поиска ключевого слова в описании вакансии
# key: search_field
# value: name, company_name или description

In [1204]:
# <div class="bloko-form-item-baseline"></div> - блок задания вывода количества вакансий на странице
# <input type="radio" name="items_on_page" value="100" data-qa="advanced-search__items_on_page-item_100" class="bloko-radio__input">
# ввод количества вакансий на странице
# key: items_on_page
# value: 100, 50 или 20

In [1205]:
# params={key: value} добавим параметры поиска
response = requests.get(url, params = { 
        'text': 'Java', 
        'search_field': 'name', 
        'items_on_page': '100', 
        'page': ''}, headers=headers)

In [1206]:
response.ok

True

In [1207]:
html = bs(response.text,'html.parser')

In [1208]:
# Зададим условие определения количество страниц с вакансиями в поисковой выдаче
pages = html.find('div', {'data-qa': 'pager-block'})

In [1209]:
pages

<div class="pager" data-qa="pager-block"><span class="" data-qa="pager-page-wrapper-1-0"><span class="bloko-button bloko-button_pressed" data-qa="pager-page"><span>1</span></span><span class="bloko-form-spacer"></span></span><span class="" data-qa="pager-page-wrapper-2-1"><a class="bloko-button" data-qa="pager-page" href="/search/vacancy?text=Java&amp;search_field=name&amp;items_on_page=100&amp;page=1&amp;hhtmFrom=vacancy_search_list" rel="nofollow"><span>2</span></a><span class="bloko-form-spacer"></span></span><span class="" data-qa="pager-page-wrapper-3-2"><a class="bloko-button" data-qa="pager-page" href="/search/vacancy?text=Java&amp;search_field=name&amp;items_on_page=100&amp;page=2&amp;hhtmFrom=vacancy_search_list" rel="nofollow"><span>3</span></a><span class="bloko-form-spacer"></span></span><span class="pager-item-not-in-short-range" data-qa="pager-page-wrapper-4-3"><a class="bloko-button" data-qa="pager-page" href="/search/vacancy?text=Java&amp;search_field=name&amp;items_on_

In [1210]:
pages.find_all('a', {'data-qa': 'pager-page'})

[<a class="bloko-button" data-qa="pager-page" href="/search/vacancy?text=Java&amp;search_field=name&amp;items_on_page=100&amp;page=1&amp;hhtmFrom=vacancy_search_list" rel="nofollow"><span>2</span></a>,
 <a class="bloko-button" data-qa="pager-page" href="/search/vacancy?text=Java&amp;search_field=name&amp;items_on_page=100&amp;page=2&amp;hhtmFrom=vacancy_search_list" rel="nofollow"><span>3</span></a>,
 <a class="bloko-button" data-qa="pager-page" href="/search/vacancy?text=Java&amp;search_field=name&amp;items_on_page=100&amp;page=3&amp;hhtmFrom=vacancy_search_list" rel="nofollow"><span>4</span></a>,
 <a class="bloko-button" data-qa="pager-page" href="/search/vacancy?text=Java&amp;search_field=name&amp;items_on_page=100&amp;page=4&amp;hhtmFrom=vacancy_search_list" rel="nofollow"><span>5</span></a>,
 <a class="bloko-button" data-qa="pager-page" href="/search/vacancy?text=Java&amp;search_field=name&amp;items_on_page=100&amp;page=17&amp;hhtmFrom=vacancy_search_list" rel="nofollow"><span>18<

In [1211]:
len(pages.find_all('a', {'data-qa': 'pager-page'}))

5

In [1212]:
pages.find_all('a', {'data-qa': 'pager-page'})[len(pages.find_all('a', {'data-qa': 'pager-page'}))-1].text

'18'

In [1213]:
int(pages.find_all('a', {'data-qa': 'pager-page'})[len(pages.find_all('a', {'data-qa': 'pager-page'}))-1].text)

18

In [1214]:
if pages is None:
    page_numbers = '1'
else:
    page_numbers = int(pages.find_all('a', {'data-qa': 'pager-page'})[len(pages.find_all('a', {'data-qa': 'pager-page'}))-1].text) 

In [1215]:
page_numbers

18

In [1216]:
vacancy_list = html.find('div', {'data-qa': 'vacancy-serp__results'}).find_all('div', {'class': 'serp-item'})
#vacancy_list

In [1217]:
len(vacancy_list)

20

In [1218]:
vacancy_search = []

for i in vacancy_list:
    
    vacancy_search_dict = {}
    vacancy_name = i.find('a', {'class': 'serp-item__title'}).getText()
    company_name = i.find('div', {'class': 'vacancy-serp-item__meta-info-company'}).getText().replace(u'\xa0', u' ')    
    company_city = i.find('div', {'data-qa': 'vacancy-serp__vacancy-address'}).getText()
    company_city = re.split(r', ', company_city)[0]
    
    # важно добавить условие нахождения <span><|span>, так как встречаются вакансии с комментарием "з/п не указана" 
    # с data-qa : "vacancy-salary-compensation-type-undefined" внутри карточки вакансии, но в списке поиска вакансий 
    # в блоке такой вакансии отсутствует <span> и если не проверять, то будет вылетать ошибка 
    
    if i.find_next('span', {'data-qa': 'vacancy-serp__vacancy-compensation' }) is None:        
        salary_min = None
        salary_max = None
        salary_currency = None
    else:
        vacancy_salary = i.find_next('span', {'data-qa': 'vacancy-serp__vacancy-compensation' }).getText().replace('\u202f', '')
        
        vacancy_salary = re.split(r'\s|-', vacancy_salary)
        
        # разбиваем строку с зарплатой на список
        # так как ЗП может указываться в вариантах 300-400 руб. (тут будет len = 4), от 300 руб. (тут будет len = 3), 
        # до 400 руб. (тут будет тоже len = 3), то нужно прописать условия,
        # при которых будут выбираться минимальное и максимальное значение и валюта (так как длина списка будет меняться)
        
        if vacancy_salary[0] == 'до':
            salary_min = None
            salary_max = int(vacancy_salary[1])
            salary_currency = vacancy_salary[2]
        elif vacancy_salary[0] == 'от':
            salary_min = int(vacancy_salary[1])
            salary_max = None
            salary_currency = vacancy_salary[2]
        else:
            salary_min = int(vacancy_salary[0])
            salary_max = int(vacancy_salary[2])            
            salary_currency = vacancy_salary[3]
        
    vacancy_link = i.find('a', {'class': 'serp-item__title'})['href']         
        
    vacancy_search_dict['vacancy_name'] = vacancy_name
    vacancy_search_dict['company_name'] = company_name
    vacancy_search_dict['company_city'] = company_city
    vacancy_search_dict['salary_min'] = salary_min
    vacancy_search_dict['salary_max'] = salary_max
    vacancy_search_dict['salary_currency'] = salary_currency
    vacancy_search_dict['vacancy_link'] = vacancy_link
    vacancy_search_dict['site'] = url.replace('/search/vacancy', '')
    vacancy_search.append(vacancy_search_dict)
vacancy_search_dict

{'vacancy_name': 'Backend-разработчик Kotlin/Java (junior)',
 'company_name': 'ООО Биллинговый центр',
 'company_city': 'Барнаул',
 'salary_min': 40000,
 'salary_max': 63000,
 'salary_currency': 'руб.',
 'vacancy_link': 'https://hh.ru/vacancy/42636573?from=vacancy_search_list&query=Java',
 'site': 'https://hh.ru'}

In [1219]:
#vacancy_search

In [1220]:
import pandas as pd

In [1221]:
df = pd.DataFrame(vacancy_search)
df

,vacancy_name,company_name,company_city,salary_min,salary_max,salary_currency,vacancy_link,site
0,Middle / Senior Java developer / Разработчик Java,МАКС СОФТ ОсОО,Бишкек,800.0,1700.0,USD,https://hh.ru/vacancy/77221563?from=vacancy_se...,https://hh.ru
1,Senior Java Developer,ЗАО Водород,Минск,50000.0,80000.0,руб.,https://hh.ru/vacancy/75884361?from=vacancy_se...,https://hh.ru
2,"Java-junior разработчик, удалённо",ООО Эврика Би Пи О,Москва,50000.0,80000.0,руб.,https://hh.ru/vacancy/76355663?from=vacancy_se...,https://hh.ru
3,Junior Java разработчик,ООО Клауд Ком,Дубна,350000.0,400000.0,руб.,https://hh.ru/vacancy/77097107?from=vacancy_se...,https://hh.ru
4,Ведущий Java разработчик (удалённо),АО НТЦ ЕЭС Информационные Комплексы,Москва,350000.0,400000.0,руб.,https://hh.ru/vacancy/76643997?from=vacancy_se...,https://hh.ru
5,Junior Java Developer,ООО Крит,Санкт-Петербург,NaN,140000.0,руб.,https://hh.ru/vacancy/75726328?from=vacancy_se...,https://hh.ru
6,Java developer (junior),ООО Диплей,Омск,40000.0,NaN,руб.,https://hh.ru/vacancy/76494200?from=vacancy_se...,https://hh.ru
7,АККРЕДИТОВАННАЯ КОМПАНИЯ Начинающий Java разра...,ЗАЗЕКС,Ростов-на-Дону,40000.0,NaN,руб.,https://hh.ru/vacancy/70667162?from=vacancy_se...,https://hh.ru
8,Senior Java developer,BrainHub,Москва,350000.0,450000.0,руб.,https://hh.ru/vacancy/76610003?from=vacancy_se...,https://hh.ru
9,Java разработчик,ЗАЗЕКС,Ростов-на-Дону,140000.0,NaN,руб.,https://hh.ru/vacancy/75697039?from=vacancy_se...,https://hh.ru


In [1222]:
def parsing_hh():

    url = 'https://hh.ru/search/vacancy'
    headers = {
                'User-Agent': 
        'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
                }
    
    vacancy_search = []
    
    params = { 
            'text': 'Java',              
            'items_on_page': '100', 
            'page': ''}

    response = requests.get(url, params = params, headers=headers)

    html = bs(response.text,'html.parser')

    # Зададим условие определения количество страниц с вакансиями в поисковой выдаче
    pages = html.find('div', {'data-qa': 'pager-block'})

    if pages is None:
        page_numbers = '1'
    else:
        page_numbers = int(pages.find_all('a', {'data-qa': 'pager-page'})[len(pages.find_all('a', {'data-qa': 'pager-page'}))-1].text)

    for page in range(0, int(page_numbers)):
        params['page'] = page
        response = requests.get(url, params = params, headers=headers)
        html = bs(response.text,'html.parser')
        vacancy_list = html.find('div', {'data-qa': 'vacancy-serp__results'}).find_all('div', {'class': 'serp-item'})
     
        for i in vacancy_list:

            vacancy_search_dict = {}

            vacancy_name = i.find('a', {'class': 'serp-item__title'}).getText()
            company_name = i.find('div', {'class': 'vacancy-serp-item__meta-info-company'}).getText().replace(u'\xa0', u' ')    
            company_city = i.find('div', {'data-qa': 'vacancy-serp__vacancy-address'}).getText()
            company_city = re.split(r', ', company_city)[0]
            
            # важно добавить условие нахождения самого тега <span>...данные по salary...<|span>, так как встречаются вакансии 
            # с комментарием "з/п не указана" с data-qa:"vacancy-salary-compensation-type-undefined" внутри карточки вакансии, 
            # но в списке поиска вакансий в блоке такой вакансии отсутствует <span> и если не проверять,то будет вылетать ошибка 
            
            if i.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation' }) is None:
                salary_min = None
                salary_max = None
                salary_currency = None
            else:
                vacancy_salary_1 = i.find_next('span', {'data-qa': 'vacancy-serp__vacancy-compensation' }) \
                .getText().replace('\u202f', '')
                vacancy_salary = re.split(r'\s|-', vacancy_salary_1)

                # разбиваем строку с зарплатой на список
                # так как ЗП может указываться в вариантах 300-400 руб. (тут будет len = 4), от 300 руб. (тут будет len = 3), 
                # до 400 руб. (тут будет тоже len = 3), то нужно прописать условия,
                # при которых будут выбираться минимальное и максимальное значение и валюта(так как длина списка будет меняться)
                
                if vacancy_salary[0] == 'до':
                    salary_min = None
                    salary_max = int(vacancy_salary[1])
                    salary_currency = vacancy_salary[2]
                elif vacancy_salary[0] == 'от':
                    salary_min = int(vacancy_salary[1])
                    salary_max = None
                    salary_currency = vacancy_salary[2]
                else:
                    salary_min = int(vacancy_salary[0])
                    salary_max = int(vacancy_salary[2])            
                    salary_currency = vacancy_salary[3]

            vacancy_link = i.find('a', {'class': 'serp-item__title'})['href']       

            vacancy_search_dict['vacancy_name'] = vacancy_name
            vacancy_search_dict['company_name'] = company_name
            vacancy_search_dict['company_city'] = company_city
            vacancy_search_dict['salary_min'] = salary_min
            vacancy_search_dict['salary_max'] = salary_max
            vacancy_search_dict['salary_currency'] = salary_currency
            vacancy_search_dict['vacancy_link'] = vacancy_link
            vacancy_search_dict['site'] = url.replace('/search/vacancy', '')
            vacancy_search.append(vacancy_search_dict)
    

    return vacancy_search
    #df = pd.DataFrame(vacancy_search)
    #return df[15:35]

In [1223]:
#parsing_hh()

### Начало работы с парсингом сайта superjob

In [1224]:
url1 = 'https://www.superjob.ru/vacancy/search/'

headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
            }

vacancy_search = []
    
params = { 
        'keywords': 'Developer',            
        'page': ''}

response = requests.get(url1, params = params, headers=headers)

html = bs(response.text,'html.parser')

In [1225]:
response.ok

True

In [1226]:
# Зададим условие определения количество страниц с вакансиями в поисковой выдаче
#pages = html.find_all('a', {'class': 'f-test-button'}) 
#pages = html.find_all('a')#.find(class_=re.compile("f-test-button"))
pages = html.find(class_=re.compile("f-test-button_active"))
pages

<a class="_1IHWd _6Nb0L _37aW8 _1isik f-test-button-1 f-test-button_active f-test-link-1" href="/vakansii/developer.html" target="_self"><span class="_28Wuq KNGBZ _4N0O3 _3lqNe _27m6C _R43B _2ixlI RcZgv" tabindex="-1"><span class="_2qWsL"><span class="_115dd">1</span></span></span></a>

In [1227]:
#pages.find_previous('div')
pages_div = pages.parent

In [1228]:
pages_div.find_all('a')

[<a class="_1IHWd _6Nb0L _37aW8 _1isik f-test-button-1 f-test-button_active f-test-link-1" href="/vakansii/developer.html" target="_self"><span class="_28Wuq KNGBZ _4N0O3 _3lqNe _27m6C _R43B _2ixlI RcZgv" tabindex="-1"><span class="_2qWsL"><span class="_115dd">1</span></span></span></a>,
 <a class="_1IHWd _6Nb0L _37aW8 _1isik f-test-button-2 f-test-link-2" href="/vakansii/developer.html?page=2" rel="next" target="_self"><span class="_28Wuq KNGBZ _4N0O3 _3lqNe _27m6C _R43B" tabindex="-1"><span class="_2qWsL"><span class="_115dd">2</span></span></span></a>,
 <a class="_1IHWd _6Nb0L _37aW8 _1isik f-test-button-3 f-test-link-3" href="/vakansii/developer.html?page=3" target="_self"><span class="_28Wuq KNGBZ _4N0O3 _3lqNe _27m6C _R43B" tabindex="-1"><span class="_2qWsL"><span class="_115dd">3</span></span></span></a>,
 <a class="_1IHWd _6Nb0L _37aW8 _1isik f-test-button-dalshe f-test-link-Dalshe" href="/vakansii/developer.html?page=2" rel="next" target="_self"><span class="_28Wuq KNGBZ _4N0O

In [1229]:
len(pages_div.find_all('a'))

4

In [1230]:
pages_div.find_all('a')[len(pages_div.find_all('a'))-2].text

'3'

In [1231]:
int(pages_div.find_all('a')[len(pages_div.find_all('a'))-2].text)

3

In [1232]:
#html

In [1233]:
vacancy_list = html.find_all('div', {'class': 'f-test-search-result-item'})
#vacancy_list

In [1234]:
vacancy_salary = vacancy_list[1].find_all('div', {'class' : 'f-test-text-company-item-salary'})
vacancy_salary

[<div class="f-test-text-company-item-salary"><span class="_2eYAG rIDaO oDIMq _33qju _3T7lp">до<!-- --> <!-- -->250 000 ₽</span>/<span class="rIDaO oDIMq _33qju _3T7lp">месяц</span></div>]

In [1235]:
vacancy_list[1].find_next('a')

<a class="_1IHWd f-test-link-Programmist_1S YrERR _2_Rn8 HyxLN" href="/vakansii/programmist-1s-45743129.html" target="_blank">Программист 1С</a>

In [1236]:
vacancy_list[8].find_next('a').find_next('a').find_next('a').getText()

'Автоматизация и консалтинг'

In [1237]:
vacancy_search = []

for i in vacancy_list:
    vacancy_search_dict = {}
    if len(i.find_all('a')) > 1: # здесь делаем исключение на контейнеры с рекламой и call to action, так как у них также
        # class : f-test-search-result-item и они попадают в нашу выборку по контейнерам с резюме
        # некоторые из них не имеют тегов <a> или имееют в количестве 1.
    
        

        # так как class не имеет стандартного названия, а название вакансии находится в первой ссылке <a>, то берем find_next, 
        # т.е. следующую за нашим родительским элементом из списка вакансий

        vacancy_name = i.find_next('a').getText()    

        # в некоторых блоках вакансий есть дополнительный блок с ссылкой на страницу работодателя и логотипом, он имеет 
        # 2-й индекс <a>, поэтому если он есть, то тег <a> c Наименование компании работодателя берем из 3-его, следующего за ним
        # если его нет, то тегов <a> внутри карточки всего два, и берем из 2-ого

        if len(i.find_all('a')) > 2:
            company_name = i.find_next('a').find_next('a').find_next('a').getText()
        else:
            company_name = i.find_next('a').find_next('a').getText()

        if i.find('span', {'class': 'f-test-text-company-item-location'}) is None:
            company_city = 'Online'
        else:
            company_city = i.find('span', {'class': 'f-test-text-company-item-location'}).findChildren()[1].getText()
            

       

        # важно добавить условие нахождения <span><|span>, так как встречаются вакансии с комментарием "з/п не указана" 
        # с data-qa : "vacancy-salary-compensation-type-undefined" внутри карточки вакансии, но в списке поиска вакансий 
        # в блоке такой вакансии отсутствует <span> и если не проверять, то будет вылетать ошибка 

        #vacancy_salary = i.find_next('div', {'class': 'f-test-text-company-item-salary' }).getText().replace(u'\xa0', u' ')

        if i.find_next('div', {'class': 'f-test-text-company-item-salary' }).getText() == 'По договорённости':        
            salary_min = None
            salary_max = None
            salary_currency = None
        else:
            vacancy_salary = i.find_next('div', {'class': 'f-test-text-company-item-salary' }).find_next('span') \
            .getText().replace(u'\xa0', u' ')

            vacancy_salary = re.split(r'\s|-', vacancy_salary)   

            # разбиваем строку с зарплатой на список
            # так как ЗП может указываться в вариантах 300-400 руб. (тут будет len = 4), от 300 руб. (тут будет len = 3), 
            # до 400 руб. (тут будет тоже len = 3), то нужно прописать условия,
            # при которых будут выбираться минимальное и максимальное значение и валюта (так как длина списка будет меняться)



            if vacancy_salary[0] == 'до':
                salary_min = None
                salary_max = int(vacancy_salary[1]+vacancy_salary[2])
                salary_currency = vacancy_salary[3]
            elif vacancy_salary[0] == 'от':
                salary_min = int(vacancy_salary[1]+vacancy_salary[2])
                salary_max = None
                salary_currency = vacancy_salary[3]         
            else:
                if len(vacancy_salary) < 4:  # условие когда ЗП указана в формате 500 000 руб., т.е. при разбивке имеем Len=3
                    # и тогда мин и макс ЗП равны
                    salary_min = int(vacancy_salary[0]+vacancy_salary[1])
                    salary_max = int(vacancy_salary[0]+vacancy_salary[1])
                    salary_currency = vacancy_salary[2]
                else:
                    salary_min = int(vacancy_salary[0]+vacancy_salary[1])
                    salary_max = int(vacancy_salary[3]+vacancy_salary[4])            
                    salary_currency = vacancy_salary[5]

        vacancy_link = url + i.find_next('a')['href']       

        vacancy_search_dict['vacancy_name'] = vacancy_name
        vacancy_search_dict['company_name'] = company_name
        vacancy_search_dict['company_city'] = company_city
        vacancy_search_dict['salary_min'] = salary_min
        vacancy_search_dict['salary_max'] = salary_max
        vacancy_search_dict['salary_currency'] = salary_currency
        vacancy_search_dict['vacancy_link'] = vacancy_link
        vacancy_search_dict['site'] = url1.replace('/vacancy/search', '')
        vacancy_search_dict['vacancy_salary'] = vacancy_salary
        vacancy_search.append(vacancy_search_dict)
    else:
        continue
vacancy_search_dict, i.find_next('a').find_next('a').find_next('a').getText(), i.find_all('a'), len(i.find_all('a'))
    

({'vacancy_name': 'Инженер-программист',
  'company_name': 'ФГБОУ ВО ТГМУ Минздрава России',
  'company_city': 'Владивосток',
  'salary_min': 20000,
  'salary_max': 30000,
  'salary_currency': '₽',
  'vacancy_link': 'https://hh.ru/search/vacancy/vakansii/inzhener-programmist-44190893.html',
  'site': 'https://www.superjob.ru/',
  'vacancy_salary': ['20', '000', '—', '30', '000', '₽']},
 'ФГБОУ ВО ТГМУ Минздрава России',
 [<a class="_1IHWd f-test-link-Inzhener-programmist YrERR _2_Rn8 HyxLN" href="/vakansii/inzhener-programmist-44190893.html" target="_blank">Инженер-программист</a>,
  <a class="_1IHWd f-test-link- YrERR" href="/clients/fgbou-vo-tgmu-minzdrava-rossii-872618.html" target="_blank"><img alt="ФГБОУ ВО ТГМУ Минздрава России" class="_285_S" loading="lazy" src="//public.superjob.ru/images/clients_logos.ru/872618_93a0af9ae7fe0f669de3045b9c782231.png"/></a>,
  <a class="_1IHWd _198Ox f-test-link-FGBOU_VO_TGMU_Minzdrava_Rossii" href="/clients/fgbou-vo-tgmu-minzdrava-rossii-872618.

In [1238]:
#vacancy_search

In [1239]:
df = pd.DataFrame(vacancy_search)
df

,vacancy_name,company_name,company_city,salary_min,salary_max,salary_currency,vacancy_link,site,vacancy_salary
0,Golang разработчик,Сбербанк,Рязань,70000.0,NaN,₽,https://hh.ru/search/vacancy/vakansii/golang-r...,https://www.superjob.ru/,"[от, 70, 000, ₽]"
1,Программист 1С,Твой Дом,Москва,NaN,250000.0,₽,https://hh.ru/search/vacancy/vakansii/programm...,https://www.superjob.ru/,"[до, 250, 000, ₽]"
2,Программист микроконтроллеров,Кадровое агентство уникальных специалистов,Москва,NaN,220000.0,₽,https://hh.ru/search/vacancy/vakansii/programm...,https://www.superjob.ru/,"[до, 220, 000, ₽]"
3,Ведущий разработчик 1С,СИАМ консалтинг,Москва,NaN,NaN,None,https://hh.ru/search/vacancy/vakansii/veduschi...,https://www.superjob.ru/,"[до, 220, 000, ₽]"
4,Программист 1С (не удаленно),FAVORIT MOTORS,Москва,250000.0,250000.0,₽,https://hh.ru/search/vacancy/vakansii/programm...,https://www.superjob.ru/,"[250, 000, ₽]"
5,Программист 1С,НИПЦ Генплана Санкт-Петербурга,Санкт-Петербург,115000.0,NaN,₽,https://hh.ru/search/vacancy/vakansii/programm...,https://www.superjob.ru/,"[от, 115, 000, ₽]"
6,Программист 1С,Автоматизация и консалтинг,Москва,150000.0,NaN,₽,https://hh.ru/search/vacancy/vakansii/programm...,https://www.superjob.ru/,"[от, 150, 000, ₽]"
7,Разработчик PHP,Ботобот,Москва,150000.0,NaN,₽,https://hh.ru/search/vacancy/vakansii/razrabot...,https://www.superjob.ru/,"[от, 150, 000, ₽]"
8,Программист баз данных,СЕВЕР-МЕТРОПОЛЬ,Санкт-Петербург,110000.0,NaN,₽,https://hh.ru/search/vacancy/vakansii/programm...,https://www.superjob.ru/,"[от, 110, 000, ₽]"
9,Программист,Сеть клиник МедЦентрСервис,Москва,130000.0,NaN,₽,https://hh.ru/search/vacancy/vakansii/programm...,https://www.superjob.ru/,"[от, 130, 000, ₽]"


In [1240]:
# Input для указания должности
# <input type="text" tallness="large" autocomplete="off" name="keywords" placeholder="Должность, компания или услуга" 
# value="" maxlength="500" class="_2kA6Y _32Oyx f-test-input-keywords _36b7h _3R0MQ _1TXqg _1q6cg _3vlg_">

In [1241]:
def parsing_superjob():

    url = 'https://www.superjob.ru/vacancy/search/'
    headers = {
                'User-Agent': 
        'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
                }
    
    vacancy_search = []
    
    params = { 
        'keywords': 'Python',            
        'page': ''}

    response = requests.get(url, params = params, headers=headers)

    html = bs(response.text,'html.parser')

    # Зададим условие определения количество страниц с вакансиями в поисковой выдаче

    pages = html.find(class_=re.compile("f-test-button_active"))
    
    #pages_div = pages.parent
    
    if pages is None:
        page_numbers = '1'
    else:
        page_numbers = int(pages.parent.find_all('a')[len(pages_div.find_all('a'))-2].text)

    for page in range(0, int(page_numbers)):
        params['page'] = page
        response = requests.get(url, params = params, headers=headers)
        html = bs(response.text,'html.parser')
        vacancy_list = html.find_all('div', {'class': 'f-test-search-result-item'})
     
        for i in vacancy_list:

            vacancy_search_dict = {}
            if len(i.find_all('a')) > 1: # здесь делаем исключение на контейнеры с рекламой и call to action, так как у них также
                # class : f-test-search-result-item и они попадают в нашу выборку по контейнерам с резюме
                # некоторые из них не имеют тегов <a> или имееют в количестве 1.



                # так как class не имеет стандартного названия, а название вакансии находится в первой ссылке <a>, то берем find_next, 
                # т.е. следующую за нашим родительским элементом из списка вакансий

                vacancy_name = i.find_next('a').getText()    

                # в некоторых блоках вакансий есть дополнительный блок с ссылкой на страницу работодателя и логотипом, он имеет 
                # 2-й индекс <a>, поэтому если он есть, то тег <a> c Наименование компании работодателя берем из 3-его, следующего за ним
                # если его нет, то тегов <a> внутри карточки всего два, и берем из 2-ого


                if len(i.find_all('a')) > 2:
                    company_name = i.find_next('a').find_next('a').find_next('a').getText()
                else:
                    company_name = i.find_next('a').find_next('a').getText()

                if i.find('span', {'class': 'f-test-text-company-item-location'}) is None:
                    company_city = 'Online'
                else:
                    company_city = i.find('span', {'class': 'f-test-text-company-item-location'}).findChildren()[1].getText()
                

                # важно добавить условие нахождения <span><|span>, так как встречаются вакансии с комментарием "з/п не указана" 
                # с data-qa : "vacancy-salary-compensation-type-undefined" внутри карточки вакансии, но в списке поиска вакансий 
                # в блоке такой вакансии отсутствует <span> и если не проверять, то будет вылетать ошибка 

                #vacancy_salary = i.find_next('div', {'class': 'f-test-text-company-item-salary' }).getText().replace(u'\xa0', u' ')

                if i.find_next('div', {'class': 'f-test-text-company-item-salary' }).getText() == 'По договорённости':        
                    salary_min = None
                    salary_max = None
                    salary_currency = None
                else:
                    vacancy_salary = i.find_next('div', {'class': 'f-test-text-company-item-salary' }).find_next('span') \
                    .getText().replace(u'\xa0', u' ')

                    vacancy_salary = re.split(r'\s|-', vacancy_salary)   

                    # разбиваем строку с зарплатой на список
                    # так как ЗП может указываться в вариантах 300-400 руб. (тут будет len = 4), от 300 руб. (тут будет len = 3), 
                    # до 400 руб. (тут будет тоже len = 3), то нужно прописать условия,
                    # при которых будут выбираться минимальное и максимальное значение и валюта (так как длина списка будет меняться)
                    
                    #vacancy_search_dict['vacancy_salary'] = vacancy_salary


                    if vacancy_salary[0] == 'до':
                        salary_min = None
                        salary_max = int(vacancy_salary[1]+vacancy_salary[2])
                        salary_currency = vacancy_salary[3]
                    elif vacancy_salary[0] == 'от':
                        salary_min = int(vacancy_salary[1]+vacancy_salary[2])
                        salary_max = None
                        salary_currency = vacancy_salary[3]         
                    else:
                        if len(vacancy_salary) < 4:  # условие когда ЗП указана в формате 500 000 руб., т.е. при разбивке имеем Len=3
                            # и тогда мин и макс ЗП равны
                            salary_min = int(vacancy_salary[0]+vacancy_salary[1])
                            salary_max = int(vacancy_salary[0]+vacancy_salary[1])
                            salary_currency = vacancy_salary[2]
                        else:
                            salary_min = int(vacancy_salary[0]+vacancy_salary[1])
                            salary_max = int(vacancy_salary[3]+vacancy_salary[4])            
                            salary_currency = vacancy_salary[5]

                vacancy_link = url[:-1] + i.find_next('a')['href']       

                vacancy_search_dict['vacancy_name'] = vacancy_name
                vacancy_search_dict['company_name'] = company_name
                vacancy_search_dict['company_city'] = company_city
                vacancy_search_dict['salary_min'] = salary_min
                vacancy_search_dict['salary_max'] = salary_max
                vacancy_search_dict['salary_currency'] = salary_currency
                vacancy_search_dict['vacancy_link'] = vacancy_link
                vacancy_search_dict['site'] = url.replace('/vacancy/search', '')                
                vacancy_search.append(vacancy_search_dict)
            else:
                continue
        vacancy_search_dict
    


    # return vacancy_search
    df = pd.DataFrame(vacancy_search)
    return df[1:50]

In [1242]:
parsing_superjob()

,vacancy_name,company_name,company_city,salary_min,salary_max,salary_currency,vacancy_link,site
1,Специалист службы поддержки с техническими зна...,Яндекс,Москва,15000.0,NaN,₽,https://www.superjob.ru/vacancy/search/vakansi...,https://www.superjob.ru/
2,Инженер-электронщик,Friendwork,Долгопрудный,100000.0,NaN,₽,https://www.superjob.ru/vacancy/search/vakansi...,https://www.superjob.ru/
3,Сервис-инженер,Интелтел,Москва,60000.0,NaN,₽,https://www.superjob.ru/vacancy/search/vakansi...,https://www.superjob.ru/
4,Руководитель аналитического отдела,Альпинтех,Москва,200000.0,NaN,₽,https://www.superjob.ru/vacancy/search/vakansi...,https://www.superjob.ru/
5,Аналитик данных,Сбербанк,Москва,NaN,NaN,None,https://www.superjob.ru/vacancy/search/vakansi...,https://www.superjob.ru/
6,Тьютор,Образовательный холдинг СИНЕРГИЯ,Москва,NaN,NaN,None,https://www.superjob.ru/vacancy/search/vakansi...,https://www.superjob.ru/
7,Системный аналитик со знанием MS SQL,Мгтс,Москва,NaN,NaN,None,https://www.superjob.ru/vacancy/search/vakansi...,https://www.superjob.ru/
8,DevOps Engineer,SuperJob,Москва,NaN,NaN,None,https://www.superjob.ru/vacancy/search/vakansi...,https://www.superjob.ru/
9,Преподаватель (доцент) на факультет информацио...,Московский политехнический университет,Москва,NaN,NaN,None,https://www.superjob.ru/vacancy/search/vakansi...,https://www.superjob.ru/
10,,Займ-Экспресс,Москва,NaN,NaN,None,https://www.superjob.ru/vacancy/search/vakansi...,https://www.superjob.ru/


In [1243]:
vacancy_list[27].find_next('a')['href']  # проверка дубль или нет

'/vakansii/inzhener-programmist-45317963.html'

In [1244]:
vacancy_list[28].find_next('a')['href'] # проверка дубль или нет

'/vakansii/programmist-45799987.html'

### Итоговые функции для парсинга HH.RU и Superjob.ru

In [1245]:
def parsing_hh():

    url = 'https://hh.ru/search/vacancy'
    headers = {
                'User-Agent': 
        'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
                }
    
    vacancy_search = []
    
    # ищем по ключевому слову Java
    
    params = { 
            'text': 'Java',              
            'items_on_page': '100', 
            'page': ''}

    response = requests.get(url, params = params, headers=headers)

    html = bs(response.text,'html.parser')

    # Зададим условие определения количество страниц с вакансиями в поисковой выдаче
    pages = html.find('div', {'data-qa': 'pager-block'}) # ищем блок с страницами

    if pages is None: # задаем условие проверки количества выдаваемых страниц с выбранной вакансией
        page_numbers = '1' # если блока страниц нет, значит вакансий меньше чем умещается на одной странице и принимаем
                           # количество страниц равным 1-ой
    else:
        page_numbers = int(pages.find_all('a', {'data-qa': 'pager-page'})[len(pages.find_all('a', {'data-qa': 'pager-page'}))-1].text)

    for page in range(0, int(page_numbers)): # задаем условие парсинга с первой страницы и далее
        params['page'] = page
        response = requests.get(url, params = params, headers=headers)
        html = bs(response.text,'html.parser')
        vacancy_list = html.find('div', {'data-qa': 'vacancy-serp__results'}).find_all('div', {'class': 'serp-item'}) 
        # задаем путь к нашим контейнерам, которые содержат контейнеры вакансий
     
        for i in vacancy_list: # и начинаем вытаскивать из каждого элемента листа вакансий, информацию по каждой вакансии
            
            vacancy_search_dict = {}

            vacancy_name = i.find('a', {'class': 'serp-item__title'}).getText()
            company_name = i.find('div', {'class': 'vacancy-serp-item__meta-info-company'}).getText().replace(u'\xa0', u' ')    
            company_city = i.find('div', {'data-qa': 'vacancy-serp__vacancy-address'}).getText()
            company_city = re.split(r', ', company_city)[0]
            
            # важно добавить условие нахождения самого тега <span>...данные по salary...<|span>, так как встречаются вакансии 
            # с комментарием "з/п не указана" с data-qa:"vacancy-salary-compensation-type-undefined" внутри карточки вакансии, 
            # но в списке поиска вакансий в блоке такой вакансии отсутствует <span> и если не проверять,то будет вылетать ошибка 
            
            if i.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation' }) is None:
                salary_min = None
                salary_max = None
                salary_currency = None
            else:
                vacancy_salary_1 = i.find_next('span', {'data-qa': 'vacancy-serp__vacancy-compensation' }) \
                .getText().replace('\u202f', '')
                vacancy_salary = re.split(r'\s|-', vacancy_salary_1)

                # разбиваем строку с зарплатой на список
                # так как ЗП может указываться в вариантах 300-400 руб. (тут будет len = 4), от 300 руб. (тут будет len = 3), 
                # до 400 руб. (тут будет тоже len = 3), то нужно прописать условия,
                # при которых будут выбираться минимальное и максимальное значение и валюта(так как длина списка будет меняться)
                
                if vacancy_salary[0] == 'до':
                    salary_min = None
                    salary_max = int(vacancy_salary[1])
                    salary_currency = vacancy_salary[2]
                elif vacancy_salary[0] == 'от':
                    salary_min = int(vacancy_salary[1])
                    salary_max = None
                    salary_currency = vacancy_salary[2]
                else:
                    salary_min = int(vacancy_salary[0])
                    salary_max = int(vacancy_salary[2])            
                    salary_currency = vacancy_salary[3]

            vacancy_link = i.find('a', {'class': 'serp-item__title'})['href']       

            vacancy_search_dict['vacancy_name'] = vacancy_name
            vacancy_search_dict['company_name'] = company_name
            vacancy_search_dict['company_city'] = company_city
            vacancy_search_dict['salary_min'] = salary_min
            vacancy_search_dict['salary_max'] = salary_max
            vacancy_search_dict['salary_currency'] = salary_currency
            vacancy_search_dict['vacancy_link'] = vacancy_link
            vacancy_search_dict['site'] = url.replace('/search/vacancy', '')
            vacancy_search.append(vacancy_search_dict)
    

    return vacancy_search

In [1246]:
def parsing_superjob():

    url = 'https://www.superjob.ru/vacancy/search/'
    headers = {
                'User-Agent': 
        'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
                }
    
    vacancy_search = []
    
    # ищем по ключевому слову Python
    params = { 
        'keywords': 'Python',             
        'page': ''}

    response = requests.get(url, params = params, headers=headers)

    html = bs(response.text,'html.parser')

    # Зададим условие определения количество страниц с вакансиями в поисковой выдаче

    pages = html.find(class_=re.compile("f-test-button_active"))
    
    #pages_div = pages.parent
    
    if pages is None:
        page_numbers = '1'
    else:
        page_numbers = int(pages.parent.find_all('a')[len(pages_div.find_all('a'))-2].text)

    for page in range(0, int(page_numbers)):
        params['page'] = page
        response = requests.get(url, params = params, headers=headers)
        html = bs(response.text,'html.parser')
        vacancy_list = html.find_all('div', {'class': 'f-test-search-result-item'})
     
        for i in vacancy_list:

            vacancy_search_dict = {}
            if len(i.find_all('a')) > 1: # здесь делаем исключение на контейнеры с рекламой и call to action, так как у них также
                # class : f-test-search-result-item и они попадают в нашу выборку по контейнерам с резюме
                # некоторые из них не имеют тегов <a> или имееют в количестве 1.



                # так как class не имеет стандартного названия, а название вакансии находится в первой ссылке <a>, то берем find_next, 
                # т.е. следующую за нашим родительским элементом из списка вакансий

                vacancy_name = i.find_next('a').getText()    

                # в некоторых блоках вакансий есть дополнительный блок с ссылкой на страницу работодателя и логотипом, он имеет 
                # 2-й индекс <a>, поэтому если он есть, то тег <a> c Наименование компании работодателя берем из 3-его, следующего за ним
                # если его нет, то тегов <a> внутри карточки всего два, и берем из 2-ого         


                if len(i.find_all('a')) > 2:
                    company_name = i.find_next('a').find_next('a').find_next('a').getText()
                else:
                    company_name = i.find_next('a').find_next('a').getText()

                if i.find('span', {'class': 'f-test-text-company-item-location'}) is None:
                    company_city = 'Online'
                else:
                    company_city = i.find('span', {'class': 'f-test-text-company-item-location'}).findChildren()[1].getText()
                    

                # важно добавить условие нахождения <span><|span>, так как встречаются вакансии с комментарием "з/п не указана" 
                # с data-qa : "vacancy-salary-compensation-type-undefined" внутри карточки вакансии, но в списке поиска вакансий 
                # в блоке такой вакансии отсутствует <span> и если не проверять, то будет вылетать ошибка 

                #vacancy_salary = i.find_next('div', {'class': 'f-test-text-company-item-salary' }).getText().replace(u'\xa0', u' ')

                if i.find_next('div', {'class': 'f-test-text-company-item-salary' }).getText() == 'По договорённости':        
                    salary_min = None
                    salary_max = None
                    salary_currency = None
                else:
                    vacancy_salary = i.find_next('div', {'class': 'f-test-text-company-item-salary' }).find_next('span') \
                    .getText().replace(u'\xa0', u' ')

                    vacancy_salary = re.split(r'\s|-', vacancy_salary)   

                    # разбиваем строку с зарплатой на список
                    # так как ЗП может указываться в вариантах 300-400 руб. (тут будет len = 4), от 300 руб. (тут будет len = 3), 
                    # до 400 руб. (тут будет тоже len = 3), то нужно прописать условия,
                    # при которых будут выбираться минимальное и максимальное значение и валюта (так как длина списка будет меняться)
                    
                    #vacancy_search_dict['vacancy_salary'] = vacancy_salary


                    if vacancy_salary[0] == 'до':
                        salary_min = None
                        salary_max = int(vacancy_salary[1]+vacancy_salary[2])
                        salary_currency = vacancy_salary[3]
                    elif vacancy_salary[0] == 'от':
                        salary_min = int(vacancy_salary[1]+vacancy_salary[2])
                        salary_max = None
                        salary_currency = vacancy_salary[3]         
                    else:
                        if len(vacancy_salary) < 4:  # условие когда ЗП указана в формате 500 000 руб., т.е. при разбивке 
                            # имеем Len=3 и тогда мин и макс ЗП равны
                            salary_min = int(vacancy_salary[0]+vacancy_salary[1])
                            salary_max = int(vacancy_salary[0]+vacancy_salary[1])
                            salary_currency = vacancy_salary[2]
                        else:
                            salary_min = int(vacancy_salary[0]+vacancy_salary[1])
                            salary_max = int(vacancy_salary[3]+vacancy_salary[4])            
                            salary_currency = vacancy_salary[5]

                vacancy_link = url[:-1] + i.find_next('a')['href']       

                vacancy_search_dict['vacancy_name'] = vacancy_name
                vacancy_search_dict['company_name'] = company_name
                vacancy_search_dict['company_city'] = company_city
                vacancy_search_dict['salary_min'] = salary_min
                vacancy_search_dict['salary_max'] = salary_max
                vacancy_search_dict['salary_currency'] = salary_currency
                vacancy_search_dict['vacancy_link'] = vacancy_link
                vacancy_search_dict['site'] = url.replace('/vacancy/search', '')                
                vacancy_search.append(vacancy_search_dict)
            else:
                continue     

    return vacancy_search    

In [1247]:
import json

def parsering_hh_sj():
    
    vacancy_search_summary = []
    vacancy_search_summary.extend(parsing_hh()) # loojing for 'Java'
    vacancy_search_summary.extend(parsing_superjob()) # looking for 'Python'
    
    df_summary = pd.DataFrame(vacancy_search_summary)

    with open('lesson_3_hh_sj.json', 'w') as f:
        json.dump(vacancy_search_summary, f)
    
    return df_summary

In [1248]:
parsering_hh_sj()

,vacancy_name,company_name,company_city,salary_min,salary_max,salary_currency,vacancy_link,site
0,Middle / Senior Java developer / Разработчик Java,МАКС СОФТ ОсОО,Бишкек,800.0,1700.0,USD,https://hh.ru/vacancy/77221563?from=vacancy_se...,https://hh.ru
1,Senior Java Developer,ЗАО Водород,Минск,NaN,NaN,None,https://hh.ru/vacancy/75884361?from=vacancy_se...,https://hh.ru
2,Инженер по автоматизированному тестированию (б...,ЦРТ | Группа компаний,Санкт-Петербург,NaN,NaN,None,https://hh.ru/vacancy/70848922?from=vacancy_se...,https://hh.ru
3,"Java-junior разработчик, удалённо",ООО Эврика Би Пи О,Москва,50000.0,80000.0,руб.,https://hh.ru/vacancy/76355663?from=vacancy_se...,https://hh.ru
4,Ведущий Java разработчик (удалённо),АО НТЦ ЕЭС Информационные Комплексы,Москва,350000.0,400000.0,руб.,https://hh.ru/vacancy/76643997?from=vacancy_se...,https://hh.ru
...,...,...,...,...,...,...,...,...
434,Эксперт по разработке BI-отчетности,Банк ВТБ (ПАО),Москва,NaN,NaN,None,https://www.superjob.ru/vacancy/search/vakansi...,https://www.superjob.ru/
435,Бизнес-аналитик в отдел управленческой отчетно...,Банк ВТБ,Москва,NaN,NaN,None,https://www.superjob.ru/vacancy/search/vakansi...,https://www.superjob.ru/
436,Портфельный риск-аналитик,Банк ВТБ,Москва,NaN,NaN,None,https://www.superjob.ru/vacancy/search/vakansi...,https://www.superjob.ru/
437,Руководитель группы оценки контрагентского риска,Банк ВТБ,Москва,NaN,NaN,None,https://www.superjob.ru/vacancy/search/vakansi...,https://www.superjob.ru/


In [1249]:
vacancy_search_summary = []
vacancy_search_summary.extend(parsing_hh())
vacancy_search_summary.extend(parsing_superjob())

In [1250]:
df_summary = pd.DataFrame(vacancy_search_summary)

In [1251]:
df_summary[50:90]

,vacancy_name,company_name,company_city,salary_min,salary_max,salary_currency,vacancy_link,site
50,Android разработчик (удаленно),ООО ИнстаДев,Санкт-Петербург,120000.0,200000.0,руб.,https://hh.ru/vacancy/77122782?from=vacancy_se...,https://hh.ru
51,Java-разработчик,ООО Центр Орбита,Москва,NaN,NaN,None,https://hh.ru/vacancy/70795624?from=vacancy_se...,https://hh.ru
52,Java разработчик (middle),АльфаСтрахование,Москва,NaN,NaN,None,https://hh.ru/vacancy/76213928?from=vacancy_se...,https://hh.ru
53,Senior Java Developer,Swordfish Security,Москва,250000.0,350000.0,руб.,https://hh.ru/vacancy/76413896?from=vacancy_se...,https://hh.ru
54,"Java Developer in Amsterdam, North Holland, Ne...",Эрней Николай,Москва,400000.0,800000.0,руб.,https://hh.ru/vacancy/77150351?from=vacancy_se...,https://hh.ru
55,Ведущий Java-разработчик,ПЭК / Офис,Краснодар,180000.0,200000.0,руб.,https://hh.ru/vacancy/71673132?from=vacancy_se...,https://hh.ru
56,"JavaEE разработчик (Wildfly, ZKoss, MySQL), Уд...","ООО HotelAdvisors, Hospitality Management & Co...",Санкт-Петербург,200000.0,220000.0,руб.,https://hh.ru/vacancy/77219066?from=vacancy_se...,https://hh.ru
57,Java разработчик,"Иннотех, Группа компаний",Самара,NaN,NaN,None,https://hh.ru/vacancy/75933745?from=vacancy_se...,https://hh.ru
58,"Java разработчик в команду ""Навыки""",ООО HeadHunter::Backend,Москва,350000.0,NaN,руб.,https://hh.ru/vacancy/76713727?from=vacancy_se...,https://hh.ru
59,Java fullstack / backend developer,ООО Айтико,Москва,200000.0,250000.0,руб.,https://hh.ru/vacancy/76734034?from=vacancy_se...,https://hh.ru
